Add the modules in the `gio` folder to our path.

In [1]:
import sys
sys.path.insert(0, "../gio")

.geo files are an old ASCII format using in mine planning software for importing well trajectory data or well log data.

These are the useful functions for importing .geo files, imported from the `gio/geo.py` module of Python code:

In [37]:
from geo import GeoFile, geo_to_lasfiles, reindex_las_index_inplace

Load the example `3162.geo` file from the `gio/data` folder.

In [3]:
test_fn = "../data/3162.geo"

In [4]:
geo = GeoFile.read(test_fn)

The geo file is parsed into a header section, and six frames.

In [9]:
geo.header

,header_param,header_value,header_descr
0,HOLE,3162,Hole
1,COMPANY,GEOEX,Company
2,LOCATION,LEIGH CREEK,Location
3,DATE_LOGGED,24/5/78,Date_Logged
4,X_LOCATION,253039,X_Location
5,Y_LOCATION,6623783,Y_Location
6,ELEV_MEAS_REF,200.18,Elev._Meas._Ref.
7,SYMBOL,CROSS,Symbol
8,DATE_MODIFIED,10-May-91 14:26:17,Date_modified


In [10]:
len(geo.frames)

6

Let's look at the first frame. Each frame is parsed as a `dict`:

In [17]:
frame = geo.frames[0]
frame

{'title': 'DENSITY',
 'comments': '    AMG ZONE: 54            EASTING: 253039.00        NORTHING: 6623783.00\n  COLLAR EL : 200.18             DATUM HEIGHT ABOVE COLLAR EL :  280 CM\n  CONTRACTOR: GEOEX PTY LTD  LOGGER:                  LOG DATE: 23-05-78\n  PROBE TYPE: DENSITY      PROBE ID:   41     READING INTERVAL: 200 MS\n                     APPARENT DEPTH:    249.0 METRE\n\n   --- RECORD LAYOUT DETAILS ---\n FIELD NO    FIELD WIDTH (BYTES)      FIELD NAME      SENSOR OFFSET (CM)\n     1              1                    FILLER\n     2              6                     DEPTH                0 CM\n     3              6                 SHORT DENSITY          135 CM\n     4              6                 LONG DENSITY           125 CM\n     5              6                   CALIPER               -2 CM\n     6              7                    FILLER',
 'constants':   header_param header_value header_descr
 0          SET      DENSITY     Set Name,
 'data': curve_name  CALIPER  LONG

In [18]:
print(frame["comments"])

    AMG ZONE: 54            EASTING: 253039.00        NORTHING: 6623783.00
  COLLAR EL : 200.18             DATUM HEIGHT ABOVE COLLAR EL :  280 CM
  CONTRACTOR: GEOEX PTY LTD  LOGGER:                  LOG DATE: 23-05-78
  PROBE TYPE: DENSITY      PROBE ID:   41     READING INTERVAL: 200 MS
                     APPARENT DEPTH:    249.0 METRE

   --- RECORD LAYOUT DETAILS ---
 FIELD NO    FIELD WIDTH (BYTES)      FIELD NAME      SENSOR OFFSET (CM)
     1              1                    FILLER
     2              6                     DEPTH                0 CM
     3              6                 SHORT DENSITY          135 CM
     4              6                 LONG DENSITY           125 CM
     5              6                   CALIPER               -2 CM
     6              7                    FILLER


In [19]:
frame["names"]

,curve_name,curve_unit,curve_data_type
0,DEPTH,CM,REAL
1,CALIPER,CM,REAL
2,LONGGAM,CM,REAL
3,SHORTGAM,CM,REAL


In [20]:
frame["data"]

curve_name,CALIPER,LONGGAM,SHORTGAM
DEPTH,,,
24911,179,25,395
24911,179,30,375
24911,179,25,375
24911,179,20,400
24911,179,20,370
...,...,...,...
-40,179,140,585
-40,179,140,585
-40,179,140,585


A more useful format is LAS files. You can easily convert a geo file into LAS files (one for each frame):

In [25]:
las_objs = geo.to_lasfiles()
las_objs

{'DENSITY': <lasio.las.LASFile at 0x16d71d79880>,
 'NEUTRON': <lasio.las.LASFile at 0x16d7224baf0>,
 'DENSITY.RUN2': <lasio.las.LASFile at 0x16d715ed940>,
 'NEUTRON.RUN2': <lasio.las.LASFile at 0x16d7224b250>,
 'DENSITY.RUN3': <lasio.las.LASFile at 0x16d71d79e20>,
 'NEUTRON.RUN3': <lasio.las.LASFile at 0x16d71d79c10>}

In [26]:
las = las_objs['DENSITY']

In [33]:
print(las.curves)

Mnemonic  Unit  Value  Description  
--------  ----  -----  -----------  
DEPTH     CM                        
CALIPER   CM                        
LONGGAM   CM                        
SHORTGAM  CM                        


In [35]:
print(las.df())

        CALIPER  LONGGAM  SHORTGAM
DEPTH                             
 24911      179       25       395
 24911      179       30       375
 24911      179       25       375
 24911      179       20       400
 24911      179       20       370
...         ...      ...       ...
-40         179      140       585
-40         179      140       585
-40         179      140       585
-40         179      140       585
-40         179      140       585

[12725 rows x 3 columns]


Notice also that the geo file does not have a regular interval in the depth index. There is a function to fix this:

In [40]:
reindex_las_index_inplace(las)

In [42]:
print(las.df())

            CALIPER     LONGGAM    SHORTGAM
DEPTH                                      
-40.000     179.000  140.000000  585.000000
-38.000     179.000  125.000000  515.000000
-36.000     180.000  130.000000  515.000000
-34.000     180.000  105.000000  540.000000
-31.999     179.001   90.010000  510.025000
...             ...         ...         ...
 24907.999  179.000   19.995007  419.965031
 24910.000  179.000   20.000000  410.000000
 24912.000  179.000   30.000000  405.000000
 24914.000      NaN         NaN         NaN
 24916.000      NaN         NaN         NaN

[12478 rows x 3 columns]


A convenience function for writing the LAS files to disc is also provided:

In [50]:
geo_to_lasfiles("../data/3162.geo", output_folder=".", reindex_las=True, convert_to_m=True)

In [51]:
from glob import glob

glob("*.las")

['3162.geo.DENSITY.las',
 '3162.geo.DENSITY.RUN2.las',
 '3162.geo.DENSITY.RUN3.las',
 '3162.geo.NEUTRON.las',
 '3162.geo.NEUTRON.RUN2.las',
 '3162.geo.NEUTRON.RUN3.las']

Clean up after myself:

In [52]:
from os import remove

for fn in glob("3162.geo.*.las"):
    remove(fn)